In [1]:
from pprint import pprint
import requests
from time import sleep
from bs4 import BeautifulSoup as bs

In [3]:
URL_hh = 'https://hh.ru/search/vacancy'
#URL_hh = 'https://hh.ru/search/vacancy?text=python&sala&area=1&ored_clusters=true&enable_snippets=true'0
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0. Safari/537.36'
headers = {'User-Agent': USER_AGENT}
params = {'enable_snippets':'true','text':'python', 'area':'1','ored_clusters':'true', 'enable_snippets':'true'}

In [4]:
def sort_salary(vac_salary_text,min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vac_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [6]:
def parse_hadH(url,headers, params, result = [], page = 0):
    params['page'] = page
    headers = headers
    response = requests.get(URL_hh, headers = headers,params=params)
    if response.status_code != 200:
        print('поиск закончен')
        return False
    dom = bs(response.text,'html.parser') 
    job_list = dom.find_all('div', {'class':'vacancy-serp-item__layout',}) #Ищем все уникальные теги, содержащие инф. о вакансиях
    #На странице помещается 20 вакансий. Если длина списка равна 20, то все найденные теги уникальны   
    if(len(job_list) <=20):
              print('все найденные теги уникальны')
    for vac in job_list:
        sleep(1)
        vac_name = vac.find('a').text
        vac_salary = vac.find('span', {'class', 'bloko-header-section-3'})
        if vac_salary:
            vac_salary = vac_salary.text
            min_salary, max_salary, currency_salary = sort_salary(vac_salary)
        else:
            min_salary, max_salary, currency_salary = None, None, None
        vac_link = vac.find('a')['href']
        result.append(   {'vac_name': vac_name,'vac_salary': vac_salary, 'min_salary': min_salary,'max_salary': max_salary,
                          'currency_salary': currency_salary,'vac_link': vac_link,'vacancy_source': 'hh.ru'})
    return result

In [8]:
#!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 kB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 1.1 MB/s eta 0:00:000:00:01m eta 0:00:01


In [9]:
from pymongo import MongoClient

In [31]:
client = MongoClient()
mydb= client.myPythonVac
myListDbVac = mydb.hhVac

In [28]:
#print(client.list_database_names())
#print(mydb.list_collection_names())
#pprint(list(mydb.users.find()))

In [33]:
listVacacies = parse_hadH(URL_hh,headers, params,page = 1) # Список вакансий на первой страице hh

все найденные теги уникальны


In [41]:
def check_unique_vac(listVacacies):
    for vacancy in listVacacies:
        if not len(list(myListDbVac.find({'vac_link': vacancy['vac_link']}))):
            myListDbVac.insert_one(vacancy) 
        

In [47]:
check_unique_vac(listVacacies)
size_db = len(list(myListDbVac.find()))
print(f'{size_db} уникадьных вакансий добавились в бд с первой страницы')

20 уникадьных вакансий добавились в бд с первой страницы
